# Actividad 2: Structured Streaming y Kafka

## Actividad NO perteneciente al Proyecto (actividad individual)

## Recuerda borrar siempre las líneas que dicen `raise NotImplementedError`

## IMPORTANTE: no se corregirá ningún notebook que no esté completamente ejecutado y muestre todas las salidas

### Punto de partida (final de la actividad 1): función `retraso_medio_periodo` modificada
#### Este ejercicio no suma puntos pero es necesario para el resto del notebook

Recordatorio: *El código que calcule esto debería ir encapsulado en una función de Python que reciba como argumento un DataFrame y devuelva como resultado el DataFrame con el cálculo indicado en el ejercicio 2 de la Actividad 1 de la asignatura*

Copia en la siguiente celda el código de tu función `retraso_medio_periodo` que has completado en el ejercicio 2 de la Actividad 1, y modifica el código de la siguiente manera.

* **Elimina el pivot**, y modifica el groupBy para incluir en él la columna `periodo_dia` por la que habías pivotado. Es decir, las dos columnas agrupadoras son `dest` y `periodo_dia`.
* Nombra la columna de agregación con el alias `avg_delay`.
* Añade dentro de la agregación otra columna adicional llamada `n_vuelos` que sea el total de filas que forman cada grupo.
* **Modifica la columna `alumno` para poner tu nombre y primer apellido (elimina los apellidos de tus compañeros de grupo de la Actividad 1)**. Puedes usar withColumn y dentro de ella, el segundo argumento será `F.lit("nombre apellido")`.
* Además, añade al DF **una nueva columna llamada `timestamp`** de valor igual a `F.current_timestamp()` (no es necesario utilizar `F.lit`). Nos servirá más adelante para ver en cuántos batches ha procesado Spark la información.

In [ ]:
from pyspark.sql import DataFrame
# no olvides otros import que te hagan falta

def retraso_medio_periodo(df: DataFrame) -> DataFrame:
    # Pega aquí el mismo código que escribiste en el ejercicio 2 de la Actividad 1,
    # modificando la columna alumno para que contenga solamente tu nombre y primer apellido, en lugar de los apellidos del grupo.
    return None

# YOUR CODE HERE
raise NotImplementedError

In [ ]:
df_prueba = spark.createDataFrame([("GRX", 1330.0, 23.0)], ["dest", "dep_time", "arr_delay"])
df_retraso = retraso_medio_periodo(df_prueba)
r = df_retraso.first()
assert(len(df_retraso.columns) == 6)
assert(all([x in df_retraso.columns for x in ["dest", "periodo_dia", "avg_delay", "n_vuelos", "alumno", "timestamp"]])) # comprobamos las columnas que debe tener
assert(r.dest == "GRX" and r.periodo_dia == "mediodia" and r.avg_delay == 23.0 and r.n_vuelos == 1 and r.alumno is not None and r.timestamp is not None)

## Instalación del paquete confluent-kafka para interactuar con Kafka desde Python sin usar Spark

In [ ]:
!pip install confluent-kafka

## Funciones auxiliares para crear y borrar el topic revisiones (completa el código con el nombre de tu cluster)

In [ ]:
from confluent_kafka import Producer
from confluent_kafka.admin import AdminClient, NewTopic
import socket

def crear_topic(admin_client: AdminClient, topic_name: str):
    """
    Crea el topic llamado topic_name con una sola partición y factor replicación 1.
    No devuelve nada pero mostrará un mensaje indicando si ha ido bien o no.
    
    :param admin_client: objeto AdminClient con el cliente de Kafka ya configurado
    :param topic_name: string con el nombre del topic que deseamos crear
    """
    topic = NewTopic(topic_name, num_partitions=1, replication_factor=1)
    fs = admin_client.create_topics([topic])
    for topic, f in fs.items():
        try:
            f.result()
            print(f"Topic {topic_name} creado")
        except Exception as e:
            print(f"Fallo al crear el topic {topic_name}: {e}")

def borrar_topic(admin_client: AdminClient, topic_name: str):
    """
    Borra un topic existente llamado topic_name que debe existir.
    No devuelve nada pero mostrará un mensaje indicando si ha ido bien o no.
    
    :param admin_client: objeto AdminClient con el cliente de Kafka ya configurado
    :param topic_name: string con el nombre del topic que deseamos borrar
    """
    fs = admin_client.delete_topics([topic_name])
    for topic, f in fs.items():
        try:
            f.result()
            print(f"Topic {topic_name} borrado")
        except Exception as e:
            print(f"Fallo al borrar el topic {topic_name}: {e}")

## Cliente para interactuar con Kafka mediante el paquete confluent-kafka, sin usar Spark

In [ ]:
nombre_cluster = None    # COMPLETA ESTA LÍNEA

# DESCOMENTA ESTAS LÍNEAS PARA CREAR EL CLIENTE CON EL NOMBRE DE TU CLUSTER

# conf = {"bootstrap.servers": f"{nombre_cluster}-w-0:9092,{nombre_cluster}-w-1:9092",
#         "sasl.mechanism": "PLAIN",
#         "client.id": socket.gethostname()}

# producer = Producer(conf)
# admin_client = AdminClient(conf)

# YOUR CODE HERE
raise NotImplementedError

### Funciones para borrar el topic, si algo hubiera ido mal y queremos repetir, y para crearlo tras haberlo borrado (o la primera vez)

In [ ]:
# borrar_topic(admin_client, "retrasos")
crear_topic(admin_client, "retrasos")

## Función para insertar datos en Kafka, en lotes de 10.000 cada 2 segundos, para dar tiempo a visualizar el resultado cambiante

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

executor = ThreadPoolExecutor(max_workers=1)

def insertar_kafka(producer: Producer, ruta_csv: str):
    """
    Inserta en Kafka los mensajes de las filas del fichero CSV en ruta_csv,
    escogiendo solo las columnas dest, dep_time y arr_delay. Cada fila es
    un mensaje formateado como JSON. 
    """

    if not os.path.isfile("flights.csv"):
        print("Trayendo flights.csv desde el bucket de Google Cloud Storage ...")
        os.system(f"hdfs dfs -copyToLocal -f {ruta_csv} .")
    
    print("Leyendo datos del CSV ...")
    vuelos_df = pd.read_csv("flights.csv")
    
    # Formateamos los mensajes como cadenas de texto con estructura de JSON con los campos dest, dep_time y arr_delay
    vuelos_dict = vuelos_df.loc[:, ["dest", "dep_time", "arr_delay"]].transpose().to_dict()
    mensajes = [str(vuelos_dict[i]).replace("'", "\"") for i in range(len(vuelos_dict))]
    
    print("Insertando mensajes en el topic retrasos ...")
    
    def insertar_bucle():
        import time
        for i, msj in enumerate(mensajes):
            producer.produce("retrasos", value=msj)
            if i % 10000 == 0:
                print(f"Se ha insertado el mensaje {i} con valor {msj}")
                producer.flush()
                time.sleep(5)
    
    # Ejecutamos de manera asíncrona todo el bucle de inserciones, para simultáneamente poder 
    # ejecutar la otra celda en la que hacemos show de la tabla de salida donde escribe Spark Structured Streaming
    resultado = executor.submit(insertar_bucle)
    return resultado

### Ejercicio 1

Utilizaremos Kafka para actualizar en tiempo real el resultado calculado en el apartado anterior.

Para simplificar, asumimos que los mensajes leídos de Kafka tiene solamente tres campos que son los únicos necesarios para llevar a cabo la operación anterior: dest, dep_time y arr_delay. La idea será crear un Streaming DataFrame para leer de Kafka, y después invocar a nuestra función `retraso_medio_periodo`, pasándolo dicho DF como argumento. Vamos a leer del topic `retrasos`, por lo que debes indicar esta opción a continuación.

Se pide crear, en la variable `retrasosStreamingDF`, un Streaming DataFrame leyendo de Apache Kafka, configurando las siguientes opciones:
  * Usar la variable `readStream` (en lugar de `read` como solemos hacer) interna de la SparkSession `spark`
  * Indicar que el formato es `"kafka"` con `.format("kafka")`
  * Indicar cuáles son los brokers de Kafka de los que vamos a leer y el puerto al que queremos conectarnos para leer (9092 es el que usa Kafka por defecto), con `.option("kafka.bootstrap.servers", "nombre_cluster-w-0:9092,nombre_cluster-w-1:9092")`. De esa manera podremos leer el mensaje si el productor de Kafka lo envía a cualquiera de los dos brokers existentes, que son los nodos del cluster identificados como `nombre_cluster-w-0` y `nombre_cluster-w-1`
  * Indicar que queremos subscribirnos al topic `"retrasos"` con `.option("subscribe", "retrasos")`.
  * Finalmente ponemos `load()` para realizar la lectura.

In [ ]:
# Esto hace más eficientes las transformaciones wide con volúmenes pequeños de datos
spark.conf.set("spark.sql.shuffle.partitions", 8)
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")

# Reemplaza por el código correcto siguiendo las indicaciones anteriores
retrasosStreamingDF = None 

# YOUR CODE HERE
raise NotImplementedError

In [ ]:
# Mostramos el esquema de este DataFrame
types = retrasosStreamingDF.dtypes
assert(retrasosStreamingDF.isStreaming)
assert((types[0][0] == "key")       & (types[0][1] == "binary"))
assert((types[1][0] == "value")     & (types[1][1] == "binary"))
assert((types[2][0] == "topic")     & (types[2][1] == "string"))
assert((types[3][0] == "partition") & (types[3][1] == "int"))
assert((types[4][0] == "offset")    & (types[4][1] == "bigint"))
assert((types[5][0] == "timestamp") & (types[5][1] == "timestamp"))
assert((types[6][0] == "timestampType") & (types[6][1] == "int"))

Muestra por pantalla el esquema del DataFrame resultante de la lectura con `printSchema()`. Verás que todas estas columnas son creadas automáticamente por Spark cuando leemos de Kafka. De ellas, la que nos interesa es `value` que contiene propiamente el mensaje de Kafka, en formato datos binarios. 

### Ejercicio 2

Tendremos que estructurar los datos leídos desde Kafka para poder extraer los campos. Para ello sigue los siguientes pasos, ayudándote de la plantilla que hay en la celda siguiente (descoméntala y complétala):

* **Selecciona** la columna `value` y conviértela (`.cast`) a `StringType()` utilizando `withColumn` para reemplazar la columna existente `"value"` por el objeto Column resultante de la conversión. De esta forma tendremos una columna que contendrá en cada **fila** un **fichero JSON completo**, tal como se muestra en cada una de las plantillas anteriores. 
* Para extraer los dos campos de cada uno de los JSON y convertirlos en una columna llamada `tuplas`, de tipo `struct` (una estructura formada por varios campos de tipos String o Double, según el campo), utilizaremos la función `from_json` de Spark, que se aplica a cada elemento (cada fila) de la columna "value" y parsea el String según un esquema que le indiquemos, devolviendo una columna de tipo `struct`.
* La columna `tuplas` es de tipo `struct` por lo que puedes acceder a cada uno de sus campos (`dest`, `dep_time` y `arr_delay`) con el operador `.` (punto). Utilizando `withColumn` dos veces, crea tres columnas llamadas `dest`, `dep_time` y `arr_delay` como el resultado de acceder a `tuplas.dest`, `tuplas.dep_time` y `tuplas.arr_delay` respectivamente.

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

esquema = StructType([
  StructField("dest", StringType()),
  StructField("dep_time", DoubleType()),
  StructField("arr_delay", DoubleType())
])

parsedDF = None   # borra esta línea y reemplaza por las siguientes líneas (descoméntalas y complétalas)

# parsedDF = <COMPLETAR>\
#     .select(<COMPLETAR)\
#     .<COMPLETAR>("value", F.col("value").cast(<COMPLETAR>))\
#     .withColumn(<COMPLETAR>, F.from_json(F.col("value"), esquema))\
#     .withColumn(<COMPLETAR>, F.col("tuplas.dest"))\
#     .withColumn(<COMPLETAR>, F.col("tuplas.dep_time"))\
#     .withColumn(<COMPLETAR>, F.col("tuplas.arr_delay"))\

# YOUR CODE HERE
raise NotImplementedError

In [ ]:
tipos = parsedDF.dtypes
assert(("value", "string") in tipos)
assert(('tuplas', 'struct<dest:string,dep_time:double,arr_delay:double>') in tipos)
assert(('dest', 'string') in tipos)
assert(('dep_time', 'double') in tipos)
assert(('arr_delay', 'double') in tipos)

* Nuestro DataFrame ya contiene una columna `dest` con el nombre del aeropuerto destino, una columna `dep_time` con la hora de salida, y una columna de números reales `arr_delay` con el retraso. 
* Por tanto, ya podemos efectuar el mismo tipo de agregación que estamos haciendo en nuestra función `retraso_medio_periodo`. 
* Para ello, invocamos a `retraso_medio_periodo` pasando como argumento `parsedDF`, que es un Streaming DataFrame porque proviene de transformar `retrasosStreamingDF` que también es un Streaming DataFrame.

In [ ]:
# Evalúa el siguiente código pero no lo modifiques
# Indicamos que este DataFrame se guarde en memoria cuando se va actualizando,
# y arrancamos la ejecución en Streaming con la acción start()

retraso_medio_streaming_df = retraso_medio_periodo(parsedDF)

consoleOutput = retraso_medio_streaming_df\
                    .writeStream\
                    .queryName("retrasosAgg")\
                    .outputMode("complete")\
                    .format("memory")\
                    .start()


## Ejercicio 3: inserción en Kafka + lectura desde Kafka con Spark y agregación en tiempo real

Una vez evaluada la celda anterior, ejecuta la celda siguiente (función `insertar_kafka`), y **cuando empiecen a aparecer mensajes indicando que se empiezan a insertar mensajes en Kafka**, entonces ejecuta (sin esperar más) la celda posterior, que contiene un bucle.
* La función `insertar_kafka`, por dentro, descargará el fichero flights.csv desde GCS al disco duro de la máquina nombrecluster-m y, después de esto, se pondrá en segundo plano a insertar mensajes en Kafka. Esto último lo hará sin bloquear el notebook, por lo que, llegado ese momento, ya podrás ejecutar la celda posterior.
* La segunda celda contiene **un código que debes completar** para leer el contenido de la vista temporal `retrasosAgg`, quedarte sólo con los vuelos cuyo destino es `JFK`, e imprimir dichas filas. Verás que, en cada iteración del bucle que contiene la celda, el resultado va cambiando, debido a que cada vez hay más vuelos en Kafka, y esto provoca que el retraso medio en cada franja del día se vaya actualizando.
* Es posible que, en algunas iteraciones, el contenido no haya cambiado con respecto a la salida previa porque no se ha llegado a refrescar. No importa. Debes asegurarte, en cualquier caso, de mostrar la salida final, que nunca cambia, lo que significa que ya se han insertado todos los mensajes en Kafka, y que Spark ya ha calculado la versión más reciente del resultado. Como no se van a insertar más mensajes una vez que se han insertado todas las filas del CSV como mensajes, el resultado de la agregación ya no volverá a cambiar.
* El test de auto-evaluación comprueba precisamente los valores finales, asumiendo que ya se han insertado todos los datos en Kafka (una sola vez).
  * Si algo va mal, no vuelvas a insertar mensajes en el topic porque entonces, el resultado final de la agregación saldrá incorrecto. Si necesitas repetir la ejecución, asegúrate de ejecutar la última celda de este notebook antes de volver a insertar nada en Kafka.
* **No se puntuará el ejercicio si no se ha imprimido por pantalla el último resultado (las 4 filas del JFK) donde se vea que los valores mostrados en pantalla coinciden con los de la celda de autoevaluación**.


In [ ]:
import time

ruta_flights_gcs = None   # ruta del fichero flights.csv en Google Cloud Storage

# resultado = insertar_kafka(producer, ruta_flights_gcs)

# YOUR CODE HERE
raise NotImplementedError

In [ ]:
# Modifica esta línea para invocar al método .sql() de la spark session, 
# en el cual debes leer de retrasosAgg y quedarte solamente con los que tienen destino JFK

agregadosDF = None   

# DESCOMENTAR EL SIGUIENTE CÓDIGO TRAS HABER RESUELTO LA LÍNEA ANTERIOR

# for i in range(15):
#     time.sleep(7)        # queremos dar tiempo a ir calculando resultados en un cluster tan pequeño
#     agregadosDF.show(truncate=False)

# Debes poder ver varias salidas distintas. Quizá no todas las 15 impresiones sean diferentes, pero muchas de ellas sí.
# Esto te muestra cómo Structured Streaming está actualizando el resultado cada vez. Probablemente las 3 o 4 últimas se
# repetirán porque ya estarán mostrando el resultado final, que no varía.

# YOUR CODE HERE
raise NotImplementedError

In [ ]:
columnas = agregadosDF.columns
assert(len(columnas) == 6)
assert(all([x in ["dest", "periodo_dia", "avg_delay", "n_vuelos", "alumno", "timestamp"] for x in columnas])) # comprobamos las columnas que debe tener agregadosDF
lista = agregadosDF.where("dest = 'JFK'").sort("dest").collect()
assert(lista[0].periodo_dia == 'maniana' and round(lista[0].avg_delay, 2) == 28.35 and lista[0].n_vuelos==356)
assert(lista[1].periodo_dia == 'tarde' and round(lista[1].avg_delay, 2) == 52.5 and lista[1].n_vuelos==8)
assert(lista[2].periodo_dia == 'noche' and round(lista[2].avg_delay, 2) == 26.82 and lista[2].n_vuelos==620)
assert(lista[3].periodo_dia == 'mediodia' and round(lista[3].avg_delay, 2) == 42.11 and lista[3].n_vuelos==333)

# Ejercicio 4: cómo se actualiza el resultado al añadir manualmente un dato más a Kafka

Vamos a insertar manualmente un nuevo mensaje en Kafka mediante el Kafka Console Producer, que puedes abrir ejecutando en una terminal (sin salir de JupyterLab) el siguiente comando. La terminal que se abre enviará datos al topic retrasos. Cada dato debes escribirlo en una línea y pulsar ENTER para que se envíe al topic retrasos.

`/usr/lib/kafka/bin/kafka-console-producer.sh --broker-list nombrecluster-w-0:9092 --topic retrasos` 

(**modifica el nombre del cluster para poner el tuyo**)

Envía el siguiente mensaje a Kafka, copiándolo (Shift + Click derecho-copiar) y pegándolo (Shift + click derecho-pegar) en el Kafka Console Producer:

`{"dest": "JFK", "dep_time": 1800.0, "arr_delay": 75.0}`

Como es un vuelo de las 18:00 horas, pertenece al período "tarde", por lo que verás que aumenta el n_vuelos, pasando de 8 a 9 y el retraso medio también, puesto que ha llegado con 75 minutos de retraso frente a los 52.5 minutos que había calculados hasta ahora de media.

In [ ]:
# Ejecuta varias veces esta celda hasta ver que ha cambiado el DF resultante
agregadosDF.show(truncate=False)
# YOUR CODE HERE
raise NotImplementedError

In [ ]:
lista = agregadosDF.collect()
assert(lista[1].periodo_dia == 'tarde' and round(lista[1].avg_delay, 2) == 55.0 and lista[1].n_vuelos==9)

## Si algo va mal y hay que repetir... ejecuta la celda siguiente

* Detén el flujo de procesamiento con `consoleOutput.stop()`
* Borra el topic `retrasos` de Kafka (puede ser necesario ejecutar varias veces la sentencia de borrado) y vuélvelo a crear.
* Borra la vista temporal `retrasosAgg`
* Vuelve a ejecutar la celda de `consoleOutput = retraso_medio_streaming_df.writeStream()...start()`
* Vuelve a ejecutar la celda de inserción en Kafka y el bucle de escrituras por pantalla.

In [ ]:
import time

consoleOutput.stop()
print("Consulta finalizada")

spark.sql("drop table retrasosAgg")
print("Vista retrasosAgg borrada")

borrar_topic(admin_client, "retrasos")
time.sleep(3)
crear_topic(admin_client, "retrasos")

## Si aun así, no se arregla ...

* Salva el notebook (botón del diskette, o File -> Save Notebook)
* En el menú de la izquierda (icono de un círculo negro con un cuadrado blanco dentro, Running Terminals and Kernels) -> Kernels -> Shut Down All
* Cierra la pestaña del notebook
* Vuelve al explorador del menú de la izquierda (icono de la carpeta) -> abre de nuevo tu notebook en tu workspace, y selecciona Kernel -> PySpark (esquina superior derecha)